Artigos pra ler: Student behaviour in unsuporvised online quizzzes : a closer look

In [1]:
import re
import os
import math
import pandas as pd
import datetime

In [2]:
path = 'database/users'

cat database/2432/399_1326.log

In [3]:
dictHorarios = {}
colunasTabela = {'Aluno':['00'],
    'Laboratorio':['00'],
    'Exercicio':['00'],
    'Horario inicial':['00'],
    'Horario final':['00'],
    'Tempo decorrido':['00']
}

df2 = pd.DataFrame(columns=['oi','sa'])
print(df2)




Empty DataFrame
Columns: [oi, sa]
Index: []


In [4]:
#Coleta a data inicial e final de cada exercicio junto com o seu horario
def coletaDatas(arquivo):
    horarios = []
    hBegin = -1
    hEnd = -1
    dataIni = -1
    dataFim = -1
    horaIni = -1
    horaFim = -1
    for linha in arquivo:
        busca = re.findall(r'\d{4}-\d{1,2}-\d{1,2} \d+:\d+:\d+\.\d+',linha)
        if(len(busca) > 0):
            horaIni = busca[0]
            horarios.append(horaIni)
    if(len(horarios)>0):
        hBegin = horarios[0]
        hEnd = horarios[-1]
    if(hBegin != -1):
        busquinha = re.findall(r'\d{4}-\d{1,2}-\d{1,2}',hBegin)
        if(len(busquinha) > 0):
            dataIni = busquinha[0]
        busquinha = re.findall(r'\d+:\d+:\d+\.\d+',hBegin)
        if(len(busquinha) > 0):
            horaIni = busquinha[0]
    if(hEnd != -1):
            busquinha = re.findall(r'\d{4}-\d{1,2}-\d{1,2}',hEnd)
            if(len(busquinha)>0):
                    dataFim = busquinha[0]
            busquinha = re.findall(r'\d+:\d+:\d+\.\d+',hEnd)
            if(len(busquinha) > 0):
                horaFim = busquinha[0]
    '''            
    print("data raw:")
    print(hBegin,hEnd)
    print(dataIni, dataFim)
    print("Datas")
    print(horaIni,horaFim)
    print("EOF")
    '''
    return horarios, dataIni, dataFim, horaIni, horaFim

In [5]:
#Calcula a quantidade de linhas de um arquivo de log
def qntd_linhas_log(arquivo):
    contador = 0
    paste = 0
    delete = 0
    for linha in arquivo.split("\n"):
        contador += 1
        busca = re.search(r'paste',linha)
        busca2 = re.search(r'delete',linha)
        busca3 = re.search(r'Backspace',linha)
        if(busca): paste += 1
        if(busca2): delete += 1
        if(busca3): delete += 1
    
    p1 = paste/contador
    p2 = delete/contador
    return contador,round(p1,2),round(p2,2)
    

In [6]:
#Calcula o tempo empenhado no exercicio
def tempoEmpenhado(data_ini, data_fim, hora_ini, hora_fim):
    s = '' + str(data_ini) + ' ' + str(hora_ini) 
    r = '' + str(data_fim) + ' ' +str(hora_fim)
    date1 = int(datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S.%f').strftime("%s"))
    date2 = int(datetime.datetime.strptime(r, '%Y-%m-%d %H:%M:%S.%f').strftime("%s"))
    difdate = date2 - date1
    return difdate

In [7]:
#Calcula a quantidade de dias em que o mesmo exercicio foi resolvido
def n_acessosColetor(arquivo):
    primeiroAcesso = 1
    difdate = 0
    for linha in arquivo:
        busca = re.search(r'\d{4}-\d{1,2}-\d{1,2}',linha)
        if(busca):
            if(primeiroAcesso):
                dataIni = busca.group(0)
                primeiroAcesso = 0
            dataAtual = busca.group(0)
            #date1 = int(datetime.datetime.strptime(dataIni, '%Y-%m-%d').strftime("%d"))
            #date2 = int(datetime.datetime.strptime(dataAtual, '%Y-%m-%d').strftime("%d"))
            #difdate = date2 - date1
            start_date = datetime.datetime.strptime(dataIni, '%Y-%m-%d')
            end_date = datetime.datetime.strptime(dataAtual, '%Y-%m-%d')
            difdate = abs(end_date-start_date)
            difdate = difdate.days
            

            
    
    return difdate

Funcoes na pasta Executions

In [8]:
#Ve a quantidade de submissoes em um exercicio
def attemptsColetor(arquivo):
    contador = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'SUBMITION',linha)
        if(busca): contador += 1
    return contador

In [9]:
#Ve a quantidade de testes em um exercicio
def testColetor(arquivo):
    contador = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'TEST',linha)
        if(busca): contador += 1
    return contador

Funcoes na pasta Scores

In [10]:
#Ve a quantidade de exercicios em um laboratorio
def exercicios_no_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'number of exercises: \d+',linha)
        if(busca): 
            busca2 = re.search(r'\d+',busca.group(0))
            return busca2.group(0)

In [11]:
#Ve a quantidade de exercicios respondidos corretamente em um laboratorio
def exer_certos_no_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'correct: \d+',linha)
        if(busca): 
            busca2 = re.search(r'\d+',busca.group(0))
            return busca2.group(0)

In [12]:
#Nota final por laboratorio
def nota_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'final score  \(0-10\): \d+\.\d+',linha)
        if(busca):
            busca2 = re.search(r'\d+\.\d+',busca.group(0))
            return busca2.group(0)

In [13]:
#Quantidade de linhas de um codigo
def qntd_linhas(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'\w*',linha)
        if(busca): qntd += 1
    return qntd

In [14]:
#Quantidade de linhas de um codigo sem comentarios etc
def qntd_linhas_filtrado(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'\S',linha)
        if(busca): 
            busca2 = re.search(r'^(?!#).*',busca.group(0))
            if(busca2): qntd += 1

    return qntd

In [63]:
#Definindo o DataFrame
colunas = ['Aluno','Lab','Exercicio','Duracao','Qntd_Linhas','Qntd_Linhas_Filtrado','Testes','Submissoes','N_Dias','PP_Paste','PP_Delete','Linhas_Log','Nota']
df = pd.DataFrame(columns = colunas)
print(df)
df = df[0:0]

Empty DataFrame
Columns: [Aluno, Lab, Exercicio, Duracao, Qntd_Linhas, Qntd_Linhas_Filtrado, Testes, Submissoes, N_Dias, PP_Paste, PP_Delete, Linhas_Log, Nota]
Index: []


In [64]:
#MAIN
for folder in os.listdir(path):
    if not folder.startswith('.'):
        
        directory = os.path.join(path,folder)+'/'
        aluno = folder
        pastaLogs = directory + '/codemirror'
        
        #Trabalhando com o arquivo do logs de interacao
        for file in os.listdir(pastaLogs):
            buscaL = re.findall(r'\d+',file)
            if(len(buscaL) > 0):
                laboratorio = buscaL[0]
                exercicio = buscaL[1]
            file = os.path.join(pastaLogs,file)
            arquivo = open(file,'r',encoding='ISO-8859-1')
            data = arquivo.read()
            dataSplitado = data.split("\n")
            #-/-
            #parte para coleta de horario
            vetHorarios, date_begin, date_end, hour_begin, hour_end = coletaDatas(dataSplitado)
            dictHorarios[str(exercicio)] = vetHorarios
            
            if(hour_begin != -1):
                #calcula o tempo emepnhado
                exr_duration = tempoEmpenhado(date_begin, date_end, hour_begin, hour_end)
            #Coleta numero de acessos
            n_acessos = n_acessosColetor(dataSplitado)
            if(n_acessos == 0): n_acessos = 1
            qt_linhas, pp_paste, pp_delete = qntd_linhas_log(data)
            df = df.append({'Aluno': aluno,'Lab' : laboratorio,'Exercicio' : exercicio,'Duracao':exr_duration
                            ,'Qntd_Linhas':0,'Qntd_Linhas_Filtrado':0,'Testes':0,'Submissoes':0,'N_Dias':n_acessos,
                            'PP_Paste':pp_paste,'PP_Delete':pp_delete,'Linhas_Log':qt_linhas,'Nota':0.0}, ignore_index=True)    

            
        #a partir de agora eu trabalho com outro tipo de arquivo para a coleta, o de tests e submissions
        pastaExecucoes = directory + '/executions'
        for file in os.listdir(pastaExecucoes):
            buscaL = re.findall(r'\d+',file)
            if(len(buscaL) > 0):
                laboratorio = buscaL[0]
                exercicio = buscaL[1]  
            file = os.path.join(pastaExecucoes,file)
            arquivo = open(file,'r',encoding='ISO-8859-1')
            data = arquivo.read()
            #Quantidade de submissoes por exercicio
            attempts = attemptsColetor(data)
            
            #Quantidade de testes por exercicio
            tests = testColetor(data)
            df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio) & (df['Exercicio'] == exercicio)
                   ,'Testes'] = tests
            df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio) & (df['Exercicio'] == exercicio)
                   ,'Submissoes'] = attempts
            
        #Coletando atributos da pasta score 
        pastaScores = directory + '/scores'
        for file in os.listdir(pastaScores):
            buscaL = re.findall(r'\d+',file)
            if(len(buscaL) > 0):
                laboratorio = buscaL[0]  
            file = os.path.join(pastaScores,file)
            arquivo = open(file,'r',encoding='ISO-8859-1')
            data = arquivo.read()
            #Quantidade de exercicios do laboratorio
            qntd_exercicios = exercicios_no_lab(data)
            #Quantidade de exercicios do laboratorio respondidos corretamente
            qntd_exer_corretos = exer_certos_no_lab(data)
            #Nota do lab
            nota_do_lab = nota_lab((data))
            if(nota_do_lab == None): nota_do_lab = 0
            df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio),'Nota'] = float(nota_do_lab)
            
        #Coletando atributos da pasta codes
        pastaCodigos = directory + '/codes'
        for file in os.listdir(pastaCodigos):
            buscaL = re.findall(r'\d+',file)
            if(len(buscaL) > 0):
                laboratorio = buscaL[0]
                exercicio = buscaL[1]  
            file = os.path.join(pastaCodigos,file)
            arquivo = open(file,'r',encoding='ISO-8859-1')
            data = arquivo.read()
            #Quantidade de linhas de um codigo
            linhas_codigo = qntd_linhas(data)
            #Quantidade de linhas de um codigo sem comentarios, importacoes
            linhas_codigo_filtrado = qntd_linhas_filtrado(data)
            df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio) & (df['Exercicio'] == exercicio)
                   ,'Qntd_Linhas'] = linhas_codigo
            df.loc[(df['Aluno'] == aluno) & (df['Lab'] == laboratorio) & (df['Exercicio'] == exercicio)
                   ,'Qntd_Linhas_Filtrado'] = linhas_codigo_filtrado
            
            


In [65]:
print("aluno:",aluno)
print("Dia inicio:",date_begin, "Dia fim:",date_end, "Hora inicio:",hour_begin,"Hora fim:", hour_end)
print("Numero de dias empenhados durante exercicio:",n_acessos)
print("Tempo empenhado na atividade em segundos:",exr_duration)
print("Numero do lab:",laboratorio,"exercicio:",exercicio,"submissoes:",attempts,"testes:",tests)
print("Total de exercicios:",qntd_exercicios,"Total respondido:",qntd_exer_corretos,"nota:",nota_do_lab)
print("Quantidade de linhas de codigo:",linhas_codigo, "Quantidade de linhas de codigo sem comentarios/importacoes:",linhas_codigo_filtrado)

aluno: 2779
Dia inicio: 2017-7-10 Dia fim: 2017-7-10 Hora inicio: 02:11:04.687 Hora fim: 02:25:57.321
Numero de dias empenhados durante exercicio: 1
Tempo empenhado na atividade em segundos: 893
Numero do lab: 536 exercicio: 1101 submissoes: 11 testes: 35
Total de exercicios: 9 Total respondido: 2 nota: 2.22222
Quantidade de linhas de codigo: 43 Quantidade de linhas de codigo sem comentarios/importacoes: 34


In [66]:
df['Duracao'] = df['Duracao'].astype(int)
df['Qntd_Linhas'] = df['Qntd_Linhas'].astype(int)
df['Qntd_Linhas_Filtrado'] = df['Qntd_Linhas_Filtrado'].astype(int)
df['Testes'] = df['Testes'].astype(int)
df['Submissoes'] = df['Submissoes'].astype(int)
df['N_Dias'] = df['N_Dias'].astype(int)
df['PP_Paste'] = df['PP_Paste'].astype(float)
df['PP_Delete'] = df['PP_Delete'].astype(float)
df['Linhas_Log'] = df['Linhas_Log'].astype(int)
df['Nota'] = df['Nota'].astype(float) 

In [67]:
#Porcentagem de exercicios corretos/ total execicios por lab



In [68]:
agrupado =  df.groupby(['Aluno','Lab']).agg({'Duracao': 'sum', 'Qntd_Linhas': 'sum', 'Qntd_Linhas_Filtrado': 'sum'
                                             , 'Testes': 'sum', 'Submissoes': 'sum', 'PP_Paste': 'sum'
                                             , 'PP_Delete': 'sum', 'Linhas_Log': 'sum', 'Nota':'mean'})

print(agrupado)

In [69]:
agrupadoMedia = df.groupby(['Aluno','Lab']).agg({'Duracao': 'mean', 'Qntd_Linhas': 'mean', 'Qntd_Linhas_Filtrado': 'mean'
                                             , 'Testes': 'mean', 'Submissoes': 'mean', 'PP_Paste': 'mean'
                                             , 'PP_Delete': 'mean', 'Linhas_Log': 'mean', 'Nota':'mean'})

print(agrupadoMedia)

In [70]:
df.to_csv('dadosBrutos.csv', encoding='utf-8', index=False)
agrupado.to_csv('dadosPorLab.csv', encoding='utf-8', index=False)
agrupadoMedia.to_csv('dadosPorLabMean.csv', encoding='utf-8', index=False)

In [ ]:
#df.loc[(df['coluna'] == 2) & (df['2'] == 4), 'coluna'] = 'novo valor'